In [1]:
# action plan
# train a distilled resnet18 using the class prototypes
# apply dmil strategy to the distilled model 
# evaluate the distilled model



In [2]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-t3gks63_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-t3gks63_
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
modelrn18 = models.resnet18(pretrained=False)

/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [5]:
import clip
import numpy as np

In [6]:
device = torch.device('cuda:1')

In [7]:
model, preprocess = clip.load("ViT-B/32",device=device)
#model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408


In [8]:
preprocess

Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(224, 224))
    <function _convert_image_to_rgb at 0x7f3b6bceb710>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [9]:
ds_train = datasets.CIFAR100(root='./data', train=True, download=True, transform=preprocess)
ds_test  = datasets.CIFAR100(root='./data', train=False, download=True, transform=preprocess)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
class_order = torch.randperm(100)

In [11]:
class_order

tensor([89,  0, 70, 84, 99, 65, 63, 60, 67, 61, 80, 87, 92,  9, 37, 35,  8, 93,
        21, 57, 53, 25, 43, 68, 88, 10, 30, 46, 39, 14, 22, 82, 83, 20, 96, 58,
         7,  5, 78, 38, 81, 71, 56, 32, 72, 31, 34, 45, 16, 33, 13, 29, 41, 49,
        86, 76, 64, 15, 94, 26, 55, 79, 52,  2, 91, 11, 17, 44, 74,  4, 24, 73,
        77, 98, 28, 95, 27, 47, 48, 42, 69, 40, 18, 36, 97,  3,  6, 85, 12,  1,
        66, 51, 62, 54, 50, 90, 75, 19, 23, 59])

In [12]:
np.unique(class_order,return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))

In [13]:
class_order[:50]

tensor([89,  0, 70, 84, 99, 65, 63, 60, 67, 61, 80, 87, 92,  9, 37, 35,  8, 93,
        21, 57, 53, 25, 43, 68, 88, 10, 30, 46, 39, 14, 22, 82, 83, 20, 96, 58,
         7,  5, 78, 38, 81, 71, 56, 32, 72, 31, 34, 45, 16, 33])

In [14]:
#write a function to retrieve only examples from the first 50 classes defined on class_order in a class subsample
def instances_from_classes(dataset, class_order):
    subset = []
    for i in range(len(dataset)):
        _, label = dataset[i]
        if label in class_order:
            subset.append(i)
    return subset


In [15]:

set_train = instances_from_classes(ds_train,class_order[:50])
set_test  = instances_from_classes(ds_test, class_order[:50])

In [16]:
dl_train = torch.utils.data.DataLoader(ds_train, batch_size=64, sampler=torch.utils.data.SubsetRandomSampler(set_train))
dl_test = torch.utils.data.DataLoader(ds_test,  batch_size=64, sampler=torch.utils.data.SubsetRandomSampler(set_test))

In [17]:
x,y = next(iter(dl_train))

In [18]:
x,y

(tensor([[[[-0.1134, -0.1134, -0.1134,  ...,  0.1347,  0.1347,  0.1347],
           [-0.1134, -0.1134, -0.1134,  ...,  0.1347,  0.1347,  0.1347],
           [-0.1134, -0.1134, -0.1134,  ...,  0.1347,  0.1347,  0.1347],
           ...,
           [ 0.3683,  0.3537,  0.3391,  ..., -0.1718, -0.1864, -0.1864],
           [ 0.3683,  0.3537,  0.3391,  ..., -0.1718, -0.1864, -0.1864],
           [ 0.3537,  0.3391,  0.3245,  ..., -0.1718, -0.1864, -0.1864]],
 
          [[ 0.7092,  0.7092,  0.7092,  ...,  1.0243,  1.0243,  1.0243],
           [ 0.7092,  0.7092,  0.7092,  ...,  1.0243,  1.0243,  1.0243],
           [ 0.7092,  0.7092,  0.7092,  ...,  1.0243,  1.0243,  1.0243],
           ...,
           [ 1.0393,  1.0393,  1.0243,  ...,  0.0038,  0.0038,  0.0038],
           [ 1.0393,  1.0393,  1.0243,  ...,  0.0038,  0.0038,  0.0038],
           [ 1.0393,  1.0393,  1.0243,  ..., -0.0112, -0.0112, -0.0112]],
 
          [[-0.8403, -0.8403, -0.8403,  ..., -0.6412, -0.6412, -0.6412],
           [-

In [19]:
modelrn18 = models.resnet18(pretrained=False)

In [20]:
modelrn18.fc = nn.Linear(512,512)

In [21]:
from tqdm import tqdm
import sklearn.metrics as metrics

In [22]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [23]:
criterion = nn.MSELoss()

In [24]:
opt = optim.Adam(modelrn18.parameters(), lr=1e-3)

In [25]:
#freeze clip parameters
for param in model.visual.parameters():
    param.requires_grad = False

In [26]:
import wandb

In [27]:
wandb.init(
    # set the wandb project where this run will be logged
    project="clip+unicom+dmil",

 
)

wandb: Currently logged in as: edson-matsubara (lablia). Use `wandb login --relogin` to force relogin


In [28]:
modelrn18.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [29]:
stop = False
loss_train = []
epoch =0
while (not stop):
    model.train()
    lloss = []
    loop = tqdm(dl_train)
    for x,y in loop:
        x = x.to(device)
        y = y.to(device)
        pred_clip = model.encode_image(x).float()
        pred_resnet = modelrn18(x).float()
        closs = criterion(pred_clip,pred_resnet)
        closs.backward()
        opt.step()
        opt.zero_grad()
        lloss.append(closs.item())
    loss_train.append(np.mean(lloss))
    epoch +=1
    wandb.log({'loss_train':loss_train[-1]})
    lloss = []
    model.eval()

 64%|█████████████████████████████████████████████▊                          | 249/391 [00:42<00:24,  5.89it/s]


 75%|██████████████████████████████████████████████████████▏                 | 294/391 [00:52<00:16,  5.79it/s]

KeyboardInterrupt: 

In [34]:
torch.save({'rn18':modelrn18.state_dict(),'class_order':class_order},'rn18_200epochs.pth')

In [35]:
model_state_dict = torch.load('rn18_200epochs.pth')

In [31]:
model2 = models.resnet18()

In [36]:
model2.fc = nn.Linear(512,512)

In [37]:
model2.load_state_dict(model_state_dict['rn18'])

<All keys matched successfully>

In [39]:
import sklearn
import sklearn.neighbors as neighbors

In [41]:
x,y = next(iter(dl_test))

In [43]:
pred = model2(x)

In [49]:
lpred = []

In [52]:
lpred = lpred + pred.tolist()

In [53]:
np.matrix(lpred).shape

(128, 512)

In [ ]:
import sklearn.metrics

In [54]:
#define sklearn test data
def get_embeddings(lmodel,dl):
    lmodel.to(device)
    loop = tqdm(dl)
    lpred = []
    ly = []
    with torch.no_grad():
        for x,y in loop:
            x = x.to(device)
            y = y.to(device)
            pred = lmodel(x)
            lpred = lpred+pred.tolist()
            ly    = ly + y.tolist()
    return lpred,ly


In [55]:
train_emb,train_y = get_embeddings(model2,dl_train)

100%|████████████████████████████████████████████████████████████████████████| 391/391 [00:47<00:00,  8.21it/s]


In [57]:
test_emb,test_y = get_embeddings(model2,dl_test)

100%|██████████████████████████████████████████████████████████████████████████| 79/79 [00:10<00:00,  7.79it/s]


In [61]:
train_emb = np.matrix(train_emb)
train_y   = np.matrix(train_y)
test_emb   = np.matrix(test_emb)
test_y     = np.matrix(test_y)


In [63]:
train_emb.shape, train_y.shape

((25000, 512), (1, 25000))

In [64]:
test_emb.shape,test_y.shape

((5000, 512), (1, 5000))

In [65]:
clf = neighbors.KNeighborsClassifier(n_neighbors=3,weights='distance')

In [67]:
clf.fit(train_emb,train_y.T)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=3, weights='distance')

In [68]:
pred = clf.predict(test_emb)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [69]:
from sklearn.metrics import classification_report

In [71]:
print(classification_report(test_y.T,pred))

              precision    recall  f1-score   support

           0       0.76      0.84      0.80       100
           5       0.52      0.43      0.47       100
           7       0.78      0.60      0.68       100
           8       0.69      0.81      0.74       100
           9       0.89      0.71      0.79       100
          10       0.57      0.30      0.39       100
          14       0.63      0.52      0.57       100
          16       0.84      0.59      0.69       100
          20       0.70      0.80      0.74       100
          21       0.73      0.69      0.71       100
          22       0.76      0.51      0.61       100
          25       0.40      0.48      0.43       100
          30       0.70      0.64      0.67       100
          31       0.67      0.63      0.65       100
          32       0.40      0.46      0.43       100
          33       0.58      0.61      0.59       100
          34       0.56      0.44      0.49       100
          35       0.73    

In [72]:
!pip install pytorch_metric_learning

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 739.9 kB/s eta 0:00:00a 0:00:01


# ok vamos tentar agora o metric learning

In [74]:
from pytorch_metric_learning import losses
loss_func = losses.SubCenterArcFaceLoss(num_classes=50,embedding_size=512)

In [75]:
x,y = next(iter(dl_train))

In [80]:
x = x.to(device)
y = y.to(device)
pred_resnet = model2(x).float()
closs = loss_func(pred_resnet,y)

RuntimeError: cuDNN error: CUDNN_STATUS_MAPPING_ERROR

In [78]:
pred_resnet.cpu()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
stop = False
loss_train = []
epoch =0
while (epoch<100):
    model.train()
    lloss = []
    loop = tqdm(dl_train)
    for x,y in loop:
        x = x.to(device)
        y = y.to(device)
        pred_resnet = modelrn18(x).float()
        closs = loss_func(pred_resnet,y)
        closs.backward()
        opt.step()
        opt.zero_grad()
        lloss.append(closs.item())
    loss_train.append(np.mean(lloss))
    epoch +=1
    wandb.log({'loss_train_dml':loss_train[-1]})
    lloss = []
    model.eval()